<a href="https://colab.research.google.com/github/rpscdev/llm/blob/main/Gemini_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install the following library for the langchanin gemini gradio
!pip install -q -U langchain langchain-google-genai faiss-cpu gradio langchain-community google-generativeai google-ai-generativelanguage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.3 MB/s eta 0:00:00


In [ ]:
import os
import gradio as gr

# LangChain specific imports for Google Gemini
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

# Api key fetching from secret tab
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
# Set it as an environment variable for LangChain

#Sample Story text for Demo
document_content = """
A lion was once sleeping in the jungle when a mouse started running up and down his body just for fun.
This disturbed the lion’s sleep, and he woke up quite angry. He was about to eat the mouse when the mouse
desperately requested the lion to set him free. “I promise you, I will be of great help to you someday if you
save me.” The lion laughed at the mouse’s confidence and let him go. One day, a six-seven hunters came into the forest
and took the lion with them. They tied him up against a tree. The lion was struggling to get out and started to whimper.
Soon, the mouse walked past and noticed the lion in trouble. Quickly, he ran and gnawed on the ropes to set the lion free.
Both of them sped off into the jungle.
"""

# Save the content to a temporary file for TextLoader to process
temp_file_path = "temp_document.txt"
with open(temp_file_path, "w") as f:
    f.write(document_content)

print(f"Document content saved to '{temp_file_path}' for processing.")

# step 1. Load and Split your text file (from the temporary file) ---
loader = TextLoader(temp_file_path)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(documents)

# step 2. Create vector store with Gemini embeddings ---
print("Creating vector store with Gemini embeddings...")
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(chunks, embedding)
print("Vector store created.")

# step 3. Setup retriever and Gemini LLM ---
print("Setting up Gemini LLM and RAG chain...")
retriever = vectorstore.as_retriever()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
print("RAG chain ready.")

# step 4. Define Gradio QnA function ---
def answer_question(query):
    try:
        # LangChain's RetrievalQA chain uses 'invoke' for new versions
        result = qa_chain.invoke({"query": query})
        return result.get('result', 'No direct answer found.') # Access 'result' key from the dictionary
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# --- Gradio interface ---
print("Launching Gradio interface...")
gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(label="Ask a question about the Story", placeholder="E.g., Type you Query?"),
    outputs=gr.Textbox(label="Gemini Answer"),
    title="Gemini-Powered RAG Q&A",
    description="Ask questions about the provided Story using Google Gemini and LangChain.",
    allow_flagging="never" # Optional: disable flagging
).launch(debug=True) # debug=True can help with troubleshooting
print("Gradio interface launched!")

Document content saved to 'temp_document.txt' for processing.
Creating vector store with Gemini embeddings...
Vector store created.
Setting up Gemini LLM and RAG chain...
RAG chain ready.
Launching Gradio interface...


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://52ce7a4d41e57ad7e4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
